# Multidimensional Wind Turbine

Many external factors can affect the power and thrust curves of wind turbines. FLORIS supports the
ability to define "multidimensional" wind turbines, where the power and thrust curves are defined
as a function of external parameters. To enable this functionality, rather than defining `power`
and `thrust_coefficient` as a function of `wind_speed` on the `power_thrust_table`, users should
instead provide a path to a data file (described below) as `power_thrust_data_file`. Additionally,
users must set the `multi_dimensional_cp_ct` option on the turbine definition to `True`.

The power thrust data file should be a CSV file with the following columns:
(`<external_parameter_1>`, `<external_parameter_2>`, ..., `ws`, `power`,
`thrust_coefficient`). The external parameters can be any relevant factors that affect the turbine
performance, and the values to be used will be specified at run time or in the FLORIS input file.
For example, the external parameters could be air density, wave height, etc. The `ws` column should
contain the wind speed values for specification of the power and thrust coefficient (stored in the
`power` and `thrust_coefficient` columns, respectively). The wind speed, power, and thrust
coefficient values should be defined for each combination of the external parameters.

The user can then specify the values of the external parameters either on the FLORIS input file
or using the `multidim_conditions` argument of `FlorisModel.set()`. The power and thrust coefficient
are determined based on the specified conditions using a nearest-neighbor approach.

The following code snippet shows an example of a multidimensional wind turbine definition and its
corresponding power thrust data file.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from floris import FlorisModel, TimeSeries

n_wind_speeds = 100
wind_speeds = np.linspace(0.1, 30, n_wind_speeds)

fmodel = FlorisModel("defaults") # Defaults to NREL 5MW turbine
fmodel.set(
    wind_data=TimeSeries(
        wind_directions=np.zeros(n_wind_speeds),
        wind_speeds=wind_speeds,
        turbulence_intensities=0.06
    ),
    layout_x=[0],
    layout_y=[0],
    wind_shear=0.0,
    turbine_type=["iea_15MW_floating_multi_dim_cp_ct"],
    reference_wind_height=-1,
)

# Now, we need to specify what external parameters to run the model for
fmodel.set(multidim_conditions={"Tp": 2, "Hs": 1})
fmodel.run()

powers = fmodel.get_turbine_powers()
thrust_coefficients = fmodel.get_turbine_thrust_coefficients()

fig, ax = plt.subplots(2, 1, sharex=True)
ax[0].plot(wind_speeds, powers, label="First condition")
ax[0].grid()
ax[0].set_ylabel("Power [kW]")
ax[1].plot(wind_speeds, thrust_coefficients)
ax[1].grid()
ax[1].set_ylabel("Thrust coefficient [-]")
ax[1].set_xlabel("Wind speed [m/s]")
ax[1].set_xlim([0, 30])

# Set a second multidimensional condition and rerun
fmodel.set(multidim_conditions={"Tp": 4, "Hs": 5})
fmodel.run()

powers = fmodel.get_turbine_powers()
thrust_coefficients = fmodel.get_turbine_thrust_coefficients()
ax[0].plot(wind_speeds, powers, label="Second condition")
ax[0].legend(loc="upper left")
ax[1].plot(wind_speeds, thrust_coefficients)

Note that this example is not meant to be represntative of a real turbine, but rather to illustrate
the functionality. At this time, FLORIS only support a single external condition combination at a
time, but multiple combinations can be run sequentially as is shown above.

As of FLORIS v4.6, users can further specify array-based multidimensional conditions, where a
different multidimensional condition can be specified for each findex. This allows users to run
multiple multidimensional condition combinations in a single call to `fmodel.run()`. Further,
multidimensional conditions can now be passed to `floris.set()` via the `WindData` objects
(`TimeSeries`, in this case).

In [ ]:
multidim_conditions = {
    "Tp": np.array([2]*n_wind_speeds + [4]*n_wind_speeds),
    "Hs": np.array([1]*n_wind_speeds + [5]*n_wind_speeds),
}

fmodel.set(
    wind_data=TimeSeries(
        wind_directions=np.zeros(n_wind_speeds*2),
        wind_speeds=np.tile(wind_speeds, 2),
        turbulence_intensities=0.06,
        multidim_conditions=multidim_conditions,
    ),
)
# Call fmodel.run() once and run both conditions in one go
fmodel.run()

powers = fmodel.get_turbine_powers()
thrust_coefficients = fmodel.get_turbine_thrust_coefficients()

fig, ax = plt.subplots(2, 1, sharex=True)
ax[0].plot(wind_speeds, powers[:n_wind_speeds], label="First condition")
ax[0].grid()
ax[0].set_ylabel("Power [kW]")
ax[1].plot(wind_speeds, thrust_coefficients[:n_wind_speeds])
ax[1].grid()
ax[1].set_ylabel("Thrust coefficient [-]")
ax[1].set_xlabel("Wind speed [m/s]")
ax[1].set_xlim([0, 30])
ax[0].plot(wind_speeds, powers[n_wind_speeds:], label="Second condition")
ax[0].legend(loc="upper left")
ax[1].plot(wind_speeds, thrust_coefficients[n_wind_speeds:])